In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Osteoporosis"
cohort = "GSE84500"

# Input paths
in_trait_dir = "../../input/GEO/Osteoporosis"
in_cohort_dir = "../../input/GEO/Osteoporosis/GSE84500"

# Output paths
out_data_file = "../../output/preprocess/Osteoporosis/GSE84500.csv"
out_gene_data_file = "../../output/preprocess/Osteoporosis/gene_data/GSE84500.csv"
out_clinical_data_file = "../../output/preprocess/Osteoporosis/clinical_data/GSE84500.csv"
json_path = "../../output/preprocess/Osteoporosis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"TGFbeta-induced switch from adipogenic to osteogenic differentiation of human mesenchymal stem cells"
!Series_summary	"Gene Expression analysis of a differentiation timeseries of human Mesenchymal Stem Cells (hMSCs) in the presence of adipogenic/osteogenic factors. hMSCs differentiate into fat cells when treated with dexamethasone (10^-6 M), insulin (10 ug/ml), rosiglitazone (10^-7 M) and IBMX (250 uM).  TGFbeta (5 ng/ml)  inhibits this process and redirects these cells to differentiate into bone cells."
!Series_summary	"Introduction: Patients suffering from osteoporosis show an increased number of adipocytes in their bone marrow, concomitant with a reduction in the pool of human mesenchymal stem cells (hMSCs) that are able to differentiate into osteoblasts, thus leading to suppressed osteogenesis."
!Series_summary	"Methods: In order be able to interfere with this process, we have investigated in vitro culture conditions whereby adipogenic differe

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Data Availability
# Examining the sample characteristics dictionary to find relevant keys for each variable

# For trait (Osteoporosis):
# The dataset doesn't directly label samples as having osteoporosis or not,
# but we can infer from the experiment design that this is about differentiation conditions
# related to osteoporosis prevention. The treatment conditions (key 2) can be used.
trait_row = 2

# For age:
# No age information is available in the sample characteristics
age_row = None

# For gender:
# No gender information is available in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value):
    """
    Convert treatment conditions to binary values representing osteogenic vs adipogenic differentiation.
    In the context of osteoporosis research, we consider treatments that promote osteogenic differentiation
    as the positive case (1) and those that don't as the negative case (0).
    """
    if isinstance(value, str) and ":" in value:
        treatment = value.split(":", 1)[1].strip().lower()
        # Based on the background info, TGFbeta inhibits adipogenic differentiation and redirects to osteogenic differentiation
        # when combined with adipogenic factors like IBMX
        if "tgfb" in treatment and "ibmx" in treatment:
            return 1  # Osteogenic differentiation (TGFbeta + IBMX promotes osteogenic)
        else:
            return 0  # Not specifically promoting osteogenic differentiation
    return None

def convert_age(value):
    """
    Convert age values to numeric (continuous) format.
    Not used in this dataset as age information is not available.
    """
    return None

def convert_gender(value):
    """
    Convert gender values to binary (0 for female, 1 for male).
    Not used in this dataset as gender information is not available.
    """
    return None

# 3. Save Metadata
# Determine if trait data is available
is_trait_available = trait_row is not None

# Validate and save cohort info for initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# If trait_row is not None, extract clinical features
if trait_row is not None:
    # Load clinical data (assuming it's been previously loaded as clinical_data)
    # Extract clinical features using the library function
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(clinical_features)
    print("Clinical Features Preview:")
    print(preview)
    
    # Save the clinical features to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical Features Preview:
{'GSM2238538': [0.0], 'GSM2238539': [0.0], 'GSM2238540': [0.0], 'GSM2238541': [0.0], 'GSM2238542': [0.0], 'GSM2238543': [0.0], 'GSM2238544': [0.0], 'GSM2238545': [0.0], 'GSM2238546': [0.0], 'GSM2238547': [0.0], 'GSM2238548': [0.0], 'GSM2238549': [0.0], 'GSM2238550': [0.0], 'GSM2238551': [0.0], 'GSM2238552': [0.0], 'GSM2238553': [1.0], 'GSM2238554': [1.0], 'GSM2238555': [1.0], 'GSM2238556': [0.0], 'GSM2238557': [0.0], 'GSM2238558': [0.0], 'GSM2238559': [0.0], 'GSM2238560': [0.0], 'GSM2238561': [0.0], 'GSM2238562': [0.0], 'GSM2238563': [0.0], 'GSM2238564': [0.0], 'GSM2238565': [1.0], 'GSM2238566': [1.0], 'GSM2238567': [1.0], 'GSM2238568': [0.0], 'GSM2238569': [0.0], 'GSM2238570': [0.0], 'GSM2238571': [0.0], 'GSM2238572': [0.0], 'GSM2238573': [0.0], 'GSM2238574': [0.0], 'GSM2238575': [0.0], 'GSM2238576': [0.0], 'GSM2238577': [1.0], 'GSM2238578': [1.0], 'GSM2238579': [1.0], 'GSM2238580': [0.0], 'GSM2238581': [0.0], 'GSM2238582': [0.0], 'GSM2238583': [0.0], 'GSM22

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers appear to be Affymetrix probe IDs (like '1007_s_at') rather than 
# standard human gene symbols (which would be like BRCA1, TP53, etc.)
# Affymetrix probe IDs need to be mapped to gene symbols for biological interpretation

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Based on observation of the gene identifiers and gene annotation data
# The 'ID' column in the gene annotation matches the probe IDs in the gene expression data
# The 'Gene Symbol' column contains the gene symbols we need to map to
prob_col = 'ID'
gene_col = 'Gene Symbol'

# 2. Get the gene mapping dataframe by extracting the relevant columns
gene_mapping = get_gene_mapping(gene_annotation, prob_col, gene_col)

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print the first few gene symbols to verify the mapping was applied
print("First 20 gene symbols after mapping:")
print(gene_data.index[:20])


First 20 gene symbols after mapping:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1',
       'A4GALT', 'A4GNT', 'AA06', 'AAAS', 'AACS', 'AACSP1', 'AADAC', 'AADACL2',
       'AADACP1', 'AADAT', 'AAED1', 'AAGAB', 'AAK1'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Create clinical features directly from clinical_data using the conversion functions defined earlier
clinical_features_df = geo_select_clinical_features(
    clinical_data, 
    trait=trait, 
    trait_row=trait_row, 
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Now link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features_df, normalized_gene_data)
print("Linked data shape:", linked_data.shape)

# Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)

# 4. Determine whether the trait and some demographic features are severely biased, and remove biased features.
# Determine whether trait is severely biased
if len(linked_data[trait].unique()) == 2:
    is_trait_biased = judge_binary_variable_biased(linked_data, trait)
else:
    is_trait_biased = judge_continuous_variable_biased(linked_data, trait)

# Print whether trait is biased or not
if is_trait_biased:
    print(f"The distribution of the feature \'{trait}\' in this dataset is severely biased.\n")
else:
    print(f"The distribution of the feature \'{trait}\' in this dataset is fine.\n")

# Handle demographic features if they exist
unbiased_linked_data = linked_data.copy()
if "Age" in unbiased_linked_data.columns:
    age_biased = judge_continuous_variable_biased(unbiased_linked_data, 'Age')
    if age_biased:
        print(f"The distribution of the feature \'Age\' in this dataset is severely biased.\n")
        unbiased_linked_data = unbiased_linked_data.drop(columns='Age')
    else:
        print(f"The distribution of the feature \'Age\' in this dataset is fine.\n")
        
if "Gender" in unbiased_linked_data.columns:
    gender_biased = judge_binary_variable_biased(unbiased_linked_data, 'Gender')
    if gender_biased:
        print(f"The distribution of the feature \'Gender\' in this dataset is severely biased.\n")
        unbiased_linked_data = unbiased_linked_data.drop(columns='Gender')
    else:
        print(f"The distribution of the feature \'Gender\' in this dataset is fine.\n")

# 5. Conduct quality check and save the cohort information.
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="This dataset contains gene expression data from human MSCs in the context of osteoporosis research, comparing osteogenic versus adipogenic differentiation conditions when treated with TGFβ and other factors."
)

# 6. If the linked data is usable, save it as a CSV file to 'out_data_file'.
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable and was not saved")

Normalized gene data saved to ../../output/preprocess/Osteoporosis/gene_data/GSE84500.csv
Clinical data saved to ../../output/preprocess/Osteoporosis/clinical_data/GSE84500.csv
Linked data shape: (54, 19846)


For the feature 'Osteoporosis', the least common label is '1.0' with 12 occurrences. This represents 22.22% of the dataset.
The distribution of the feature 'Osteoporosis' in this dataset is fine.



Linked data saved to ../../output/preprocess/Osteoporosis/GSE84500.csv
